In [1]:
import os
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import numpy as np
from ribbn_scripts.processing.get_offset import get_offset
from ribbn_scripts.processing.phase_estimate import multidist_multifreq_phase_estimation
from ribbn_scripts.processing.localization import estimate_dph_df,direct_distance_estimate,estimate_k,unwrap_phases
from ribbn_scripts.ref_functions.spec_functions import read_network_analyzer_file, get_theta, get_amplitude, s2z
import pickle

In [15]:
# loading notes csv
labels = pd.read_csv('labels.csv',delimiter="\t")

In [16]:
labels

,Run,Label,T2T distance,Wall dist
0,0,No human,83.6,70
1,1,No human,83.6,70
2,2,No human,83.6,70
3,3,Very close,83.6,70
4,4,Very close,83.6,70
5,5,Very close,83.6,70
6,6,Bit further. In front of exciter.,83.6,70
7,7,Bit further. In front of exciter.,83.6,70
8,8,Bit further. In front of exciter.,83.6,70
9,9,"Bit further, close to one tag",83.6,70


In [19]:
labels.columns

Index(['Run', 'Label', 'T2T distance', 'Wall dist'], dtype='object')

In [24]:
df_all=None
file_names = sorted([f for f in os.listdir("dataframes") if f[0] != '.'], key=lambda x: int(x.split('.')[0]))
for file_name in file_names:
    dist_idx=int(file_name.split(".")[0])
    T2Tdist=float(labels[labels['Run']==dist_idx]['T2T distance'].iloc[0])
    label=labels[labels['Run']==dist_idx]['Label'].iloc[0]
    wallDist=float(labels[labels['Run']==dist_idx]['Wall dist'].iloc[0])
    with open(f'dataframes/{file_name}', "rb") as f:
        print(file_name)
        _df = pickle.load(f)
        # Add new columns to _df for all rows
        _df['Run Exp Num'] = dist_idx
        _df['dist'] = T2Tdist/100
        _df['label']=label
        _df['wall dist']=wallDist
        # _df['wall dist'] = wallDist/100
        if df_all is None:
            df_all=_df
        else:
            df_all = pd.concat([df_all, _df], ignore_index=True)

df_all

0.df
1.df
2.df
3.df
4.df
5.df
6.df
7.df
8.df
9.df
10.df
11.df


,Rx,Tx,MPP Start Time (s),MPP Stop Time (s),Voltages (mV),Frequency (MHz),Run Exp Num,NumMPPs,dist,label,wall dist
0,Tag1,Tag2,1.760217e+09,1.760217e+09,"[71.76, 72.59, 72.75, 72.82, 72.82, 72.67, 72....",915,0,5,0.836,No human,70.0
1,Tag2,Tag1,1.760217e+09,1.760217e+09,"[222.06, 219.54, 219.69, 219.54, 220.15, 220.0...",915,0,5,0.836,No human,70.0
2,Tag1,Tag2,1.760218e+09,1.760218e+09,"[76.18, 77.1, 77.4, 77.25, 77.33, 77.33, 77.48...",915,1,5,0.836,No human,70.0
3,Tag2,Tag1,1.760218e+09,1.760218e+09,"[223.28, 220.15, 219.69, 220.0, 219.77, 220.15...",915,1,5,0.836,No human,70.0
4,Tag1,Tag2,1.760219e+09,1.760219e+09,"[74.81, 75.65, 75.57, 75.65, 75.27, 75.42, 75....",915,2,5,0.836,No human,70.0
5,Tag2,Tag1,1.760219e+09,1.760219e+09,"[225.34, 222.29, 222.82, 222.36, 222.67, 222.4...",915,2,5,0.836,No human,70.0
6,Tag1,Tag2,1.760218e+09,1.760218e+09,"[74.5, 110.44, 110.28, 110.44, 110.51, 109.83,...",915,3,5,0.836,Very close,70.0
7,Tag2,Tag1,1.760218e+09,1.760218e+09,"[223.96, 180.86, 180.93, 181.16, 181.47, 180.4...",915,3,5,0.836,Very close,70.0
8,Tag1,Tag2,1.760218e+09,1.760218e+09,"[77.48, 97.24, 97.24, 97.31, 97.47, 97.62, 97....",915,4,5,0.836,Very close,70.0
9,Tag2,Tag1,1.760218e+09,1.760218e+09,"[223.2, 199.17, 199.09, 199.09, 199.24, 199.7,...",915,4,5,0.836,Very close,70.0


In [27]:
time_per_phase = 10/1000  # s
num_phases=6
num_mpps=df_all.iloc[0]['NumMPPs']
sampling_rate=1000
plotting=False
ver_lines = [0] + [ (time_per_phase-time_per_phase*0.01) * sampling_rate * (i + 1) for i in range(num_phases*num_mpps)]

processedDF=pd.DataFrame(columns=["Rx","Tx", "Voltages (mV)", "Phase1","Phase3","Phase4","Phase6","Phase7","Phase8",
                                "Frequency (MHz)", "Run Exp Num", "NumMPPs"])
processedDF_aggregated=pd.DataFrame(columns=["Rx","Tx","phase", "median", "std","freq", "dist", 'delta', "Experiment Number", "Unique Exp Number"])

if plotting:
    plt.figure(figsize=(15,20))

unique_exp_no=0
phase_order=[1,3,4,6,7,8]

for df_idx in range(len(df_all)):

    voltages=df_all.iloc[df_idx]['Voltages (mV)']
    phase_medians={
        1:[],
        3:[],
        4:[],
        6:[],
        7:[],
        8:[],
    }
    for idx,v in enumerate(ver_lines):
        if idx<len(ver_lines)-1:
            phase_medians[phase_order[int(idx%num_phases)]].append(np.median(voltages[int(ver_lines[idx]):int(ver_lines[idx+1])]))   
        if plotting:
            plt.subplot(len(df_all)//3+1, 3, df_idx+1)
            if idx%num_phases==0:
                plt.axvline(x=v, color='b', linestyle='-')
            else:
                plt.axvline(x=v, color='r', linestyle='--')
    entry={
        "Rx": df_all.iloc[df_idx]["Rx"],
        "Tx":df_all.iloc[df_idx]["Tx"],
        "Voltages (mV)":df_all.iloc[df_idx]["Voltages (mV)"],
        "Phase1":phase_medians[1],
        "Phase3":phase_medians[3],
        "Phase4":phase_medians[4],
        "Phase6":phase_medians[6],
        "Phase7":phase_medians[7],
        "Phase8":phase_medians[8],
        "Frequency (MHz)":df_all.iloc[df_idx]["Frequency (MHz)"],
        "Run Exp Num":df_all.iloc[df_idx]["Run Exp Num"],
        "Dist (m)":df_all.iloc[df_idx]["dist"],
        "Wall Dist (m)":df_all.iloc[df_idx]["wall dist"],
        "label": df_all.iloc[df_idx]["label"],
        "NumMPPs":df_all.iloc[df_idx]["NumMPPs"],
    }
    for phase in phase_order:
        entry_aggregated={
            "Rx":df_all.iloc[df_idx]["Rx"],
            "Tx":df_all.iloc[df_idx]["Tx"],
            "phase": str(phase),
            "median": np.mean(phase_medians[phase]), # mean of medians
            "std": np.std(phase_medians[phase]),
            "freq": df_all.iloc[df_idx]["Frequency (MHz)"]*1e6,
            "dist":df_all.iloc[df_idx]["dist"],
            "wall dist":df_all.iloc[df_idx]["wall dist"],
            "label":df_all.iloc[df_idx]["label"],
            'delta': max(phase_medians[phase])-min(phase_medians[phase]),
            "Experiment Number":df_all.iloc[df_idx]["Run Exp Num"],
            "Unique Exp Number":unique_exp_no,
            "allVoltages": phase_medians[phase]
        }
        processedDF_aggregated=pd.concat([processedDF_aggregated, pd.DataFrame([entry_aggregated])], ignore_index=True)
    unique_exp_no+=1
    
    # Identify and print bad MPPs
    bad=False
    for k in phase_medians.keys():
        assert(len(phase_medians[k])==num_mpps)
        if np.std(phase_medians[k])>1:
            d=df_all.iloc[df_idx]["wall dist"]
            print(f"<== Bad data found at {df_idx} df_idx at phase {k} at T2T dist {d}.", end="\t")
            print(f"Num vals: {len(phase_medians[k])}, mean: {np.mean(phase_medians[k])}, stdev: {np.std(phase_medians[k])} ==>")
    
    processedDF=pd.concat([processedDF,pd.DataFrame([entry])],ignore_index=True)
    if plotting:
        plt.plot(voltages,'.')
        plt.title(df_idx)
        plt.tight_layout()
        
if plotting:
    plt.show()
    

    

/var/folders/hm/7p8fzz2x2y10pbydpxcs7vw00000gn/T/ipykernel_93433/3838254931.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processedDF_aggregated=pd.concat([processedDF_aggregated, pd.DataFrame([entry_aggregated])], ignore_index=True)


In [28]:
with open("processedDF.pkl", 'wb') as f:
    pickle.dump(processedDF,f)
processedDF

,Rx,Tx,Voltages (mV),Phase1,Phase3,Phase4,Phase6,Phase7,Phase8,Frequency (MHz),Run Exp Num,NumMPPs,Dist (m),Wall Dist (m),label
0,Tag1,Tag2,"[71.76, 72.59, 72.75, 72.82, 72.82, 72.67, 72....","[72.67, 72.555, 72.555, 72.44, 72.36500000000001]","[72.1, 72.36500000000001, 72.25, 72.21, 72.175]","[70.27000000000001, 70.27000000000001, 70.2700...","[71.795, 71.64, 71.91, 71.68, 71.83]","[72.02000000000001, 71.91, 72.06, 72.06, 72.175]","[72.47999999999999, 72.59, 72.405, 72.33000000...",915,0,5,0.836,70.0,No human
1,Tag2,Tag1,"[222.06, 219.54, 219.69, 219.54, 220.15, 220.0...","[219.69, 219.62, 219.54000000000002, 219.655, ...","[220.3, 220.11, 220.23, 220.49, 220.375]","[222.785, 222.63, 222.97, 222.97, 222.9]","[222.67, 222.4, 222.66500000000002, 222.325, 2...","[222.13, 222.13, 222.095, 222.06, 222.36]","[221.75, 221.79000000000002, 221.52, 221.75, 2...",915,0,5,0.836,70.0,No human
2,Tag1,Tag2,"[76.18, 77.1, 77.4, 77.25, 77.33, 77.33, 77.48...","[77.33, 77.785, 77.36500000000001, 77.55, 77.3...","[76.985, 77.1, 76.79, 76.985, 77.02]","[74.96, 75.22999999999999, 75.0, 75.19, 75.04]","[76.75, 76.715, 76.56, 76.79, 76.565]","[77.13499999999999, 76.94, 77.02, 77.06, 77.02]","[77.48, 77.33, 77.405, 77.4, 77.4]",915,1,5,0.836,70.0,No human
3,Tag2,Tag1,"[223.28, 220.15, 219.69, 220.0, 219.77, 220.15...","[220.0, 220.23, 220.075, 220.19, 220.035]","[220.38, 220.53, 220.265, 220.875, 220.57]","[223.505, 223.54500000000002, 223.315, 223.13,...","[223.165, 223.125, 223.2, 222.935, 222.86]","[222.63, 222.74, 222.78, 222.82, 222.44]","[222.25, 222.13, 222.13, 222.28500000000003, 2...",915,1,5,0.836,70.0,No human
4,Tag1,Tag2,"[74.81, 75.65, 75.57, 75.65, 75.27, 75.42, 75....","[75.65, 75.65, 75.65, 75.65, 75.61]","[75.34, 75.305, 75.49, 75.15, 75.27]","[73.21, 73.245, 73.36, 73.21, 73.13]","[74.88, 74.81, 75.075, 75.04, 75.11]","[74.96, 75.11, 75.34, 75.19, 75.27]","[75.65, 75.61, 75.65, 75.49, 75.75999999999999]",915,2,5,0.836,70.0,No human
5,Tag2,Tag1,"[225.34, 222.29, 222.82, 222.36, 222.67, 222.4...","[222.67, 222.555, 222.515, 222.51, 222.63]","[222.78, 222.935, 222.935, 222.935, 222.9]","[225.49, 225.57, 225.64499999999998, 225.95, 2...","[225.3, 225.33499999999998, 225.265, 225.15, 2...","[224.69, 224.96, 225.03, 224.995, 225.03]","[224.73, 224.42, 224.61, 224.65, 224.535]",915,2,5,0.836,70.0,No human
6,Tag1,Tag2,"[74.5, 110.44, 110.28, 110.44, 110.51, 109.83,...","[110.28, 110.245, 110.17, 110.245, 110.0200000...","[109.78999999999999, 109.71000000000001, 109.6...","[107.16, 107.27000000000001, 107.39, 107.08, 1...","[109.22, 109.37, 109.33000000000001, 109.41, 1...","[109.715, 109.75, 109.83, 109.67, 109.6]","[110.06, 109.98, 110.245, 109.78999999999999, ...",915,3,5,0.836,70.0,Very close
7,Tag2,Tag1,"[223.96, 180.86, 180.93, 181.16, 181.47, 180.4...","[181.09, 181.28, 181.47, 181.47, 181.24]","[181.85, 182.42000000000002, 182.005, 182.15, ...","[185.475, 185.435, 185.44, 185.66, 185.4]","[185.015, 184.9, 185.09, 184.865, 184.935]","[184.70999999999998, 184.6, 184.67, 184.635, 1...","[184.175, 184.21499999999997, 184.025, 183.985...",915,3,5,0.836,70.0,Very close
8,Tag1,Tag2,"[77.48, 97.24, 97.24, 97.31, 97.47, 97.62, 97....","[97.24, 97.58000000000001, 98.685, 98.76, 99.15]","[97.275, 97.66, 98.535, 98.76, 99.03]","[95.18, 95.56, 96.44, 96.86, 97.01]","[96.4, 96.63, 97.735, 98.11500000000001, 98.0]","[96.7, 97.7, 98.03999999999999, 98.5, 98.5]","[97.31, 98.5, 98.5, 99.11, 98.99]",915,4,5,0.836,70.0,Very close
9,Tag2,Tag1,"[223.2, 199.17, 199.09, 199.09, 199.24, 199.7,...","[199.24, 199.55, 199.015, 199.17, 199.28]","[200.01, 200.005, 199.70499999999998, 200.0450...","[202.45, 202.52499999999998, 202.37, 202.3, 20...","[202.53, 202.18, 202.26, 201.95499999999998, 2...","[201.95499999999998, 201.76, 201.8399999999999...","[201.42000000000002, 201.41500000000002, 201.0...",915,4,5,0.836,70.0,Very close


In [29]:
with open("processedDF_aggregated.pkl", 'wb') as f:
    pickle.dump(processedDF_aggregated,f)
processedDF_aggregated

,Rx,Tx,phase,median,std,freq,dist,delta,Experiment Number,Unique Exp Number,wall dist,label,allVoltages
0,Tag1,Tag2,1,72.517,0.105195,915000000.0,0.836,0.305,0,0,70.0,No human,"[72.67, 72.555, 72.555, 72.44, 72.36500000000001]"
1,Tag1,Tag2,3,72.220,0.087693,915000000.0,0.836,0.265,0,0,70.0,No human,"[72.1, 72.36500000000001, 72.25, 72.21, 72.175]"
2,Tag1,Tag2,4,70.216,0.074726,915000000.0,0.836,0.190,0,0,70.0,No human,"[70.27000000000001, 70.27000000000001, 70.2700..."
3,Tag1,Tag2,6,71.771,0.098813,915000000.0,0.836,0.270,0,0,70.0,No human,"[71.795, 71.64, 71.91, 71.68, 71.83]"
4,Tag1,Tag2,7,72.045,0.085088,915000000.0,0.836,0.265,0,0,70.0,No human,"[72.02000000000001, 71.91, 72.06, 72.06, 72.175]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Tag2,Tag1,3,252.095,0.196214,915000000.0,0.836,0.535,11,23,70.0,"Bit further, close to one tag","[251.96, 251.815, 252.35, 252.07999999999998, ..."
140,Tag2,Tag1,4,256.214,0.159261,915000000.0,0.836,0.425,11,23,70.0,"Bit further, close to one tag","[256.235, 256.045, 256.04499999999996, 256.47,..."
141,Tag2,Tag1,6,255.641,0.215323,915000000.0,0.836,0.645,11,23,70.0,"Bit further, close to one tag","[255.7, 255.285, 255.55, 255.93, 255.74]"
142,Tag2,Tag1,7,255.139,0.117745,915000000.0,0.836,0.345,11,23,70.0,"Bit further, close to one tag","[255.17, 255.09, 254.94, 255.20999999999998, 2..."


In [30]:
lambda_ = 3e8/915e6
th_phi = 2*np.pi*(112.2/100)/lambda_
th_phi = th_phi % (np.pi)

In [31]:
voltage_txT1={}
voltage_txT2={}
for idx in range(processedDF_aggregated.shape[0]):
    wall_dist=processedDF_aggregated.iloc[idx]["wall dist"]
    voltages=processedDF_aggregated.iloc[idx]["allVoltages"]
    if processedDF_aggregated.iloc[idx]["Tx"]=="Tag1":
        if wall_dist not in voltage_txT1:
            voltage_txT1[wall_dist]=np.array([])
        voltage_txT1[wall_dist]=np.concat([voltage_txT1[wall_dist], np.array(voltages)])
    else:
        if wall_dist not in voltage_txT2:
            voltage_txT2[wall_dist]=np.array([])
        voltage_txT2[wall_dist]=np.concat([voltage_txT2[wall_dist], np.array(voltages)])
        
avg_voltage_txT1={}
avg_voltage_txT2={}

for k in voltage_txT1.keys():
    avg_voltage_txT1[k]=np.mean(voltage_txT1[k])
    avg_voltage_txT2[k]=np.mean(voltage_txT2[k])

In [32]:
# freq_range=np.array(range(775,1000,10))*1e6
freq_range=np.array([915])*1e6
all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, all_errors, \
    selected_experiments_all = multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=0,plot=False)

with open("no_correction_all_errs.pkl",'wb') as f:
    pickle.dump(all_errors,f)
    
with open("no_correction_all_phases.pkl",'wb') as f:
    pickle.dump(all_freqs,f)

with open("no_correction_all_phases_unwrapped.pkl",'wb') as f:
    pickle.dump(all_freqs_unwrapped,f)    

with open("theoretical_phase.pkl",'wb') as f:
    pickle.dump(all_freqs_theoretical,f)    

In [33]:
x=[]
for i in processedDF_aggregated["wall dist"].unique():
    x+=[i]*5

### With offset correction using the ground truth.

In [35]:
# correction_offsets={}
# all_errors=None


# all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, _,_=\
#     multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=0,plot=False)


# for freq in freq_range:
#     offset,best_error,errors=get_offset(all_freqs[freq],freq)
#     # print(f"Freq {freq//1e6};\t Mean:{np.mean(errors)};\t Median:{np.median(errors)};\t Std:{np.std(errors)}")
#     # if all_errors is None:
#     #     all_errors=errors
#     # else:
#     #     all_errors=np.concat((all_errors,errors))
#     correction_offsets[freq]=offset



# all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, all_errors,_=\
#     multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=correction_offsets,plot=True)


# with open("gt_correction_all_errs.pkl",'wb') as f:
#     pickle.dump(all_errors,f)
    
# with open("gt_correction_all_phases.pkl",'wb') as f:
#     pickle.dump(all_freqs,f)

# with open("gt_correction_all_phases_unwrapped.pkl",'wb') as f:
#     pickle.dump(all_freqs_unwrapped,f)    

### With correction offsets from pretrained

In [36]:
correction_offsets=pickle.load(open("/Users/manavjeet/git/T2TExperiments/DistExperiments/correction_offsets.pkl",'rb'))

all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, all_errors,_=\
    multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=correction_offsets,plot=False)


with open("pretrained_correction_all_errs.pkl",'wb') as f:
    pickle.dump(all_errors,f)
    
with open("pretrained_correction_all_phases.pkl",'wb') as f:
    pickle.dump(all_freqs,f)

with open("pretrained_correction_all_phases_unwrapped.pkl",'wb') as f:
    pickle.dump(all_freqs_unwrapped,f)